# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

In [ ]:
#Instalar selenium
#Download do chromedriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Chrome()

#Entrar no google - passo 1
navegador.get("https://www.google.com/")

#Pesquisar cotação dólar - passo 2
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dólar')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#Pegar cotação dólar - passo 3
cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)

#Pegar cotação euro - passo 4
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

#Pegar cotação ouro - passo 5 
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace("," , ".")
print(cotacao_ouro)



In [21]:
#Importar base de dados - passo 6
import pandas as pd
import numpy as np

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [42]:
#Atualizar base de dados - passo 7

#dolar
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)

#euro
tabela.loc[tabela["Moeda"]=="Euro", "Cotação"] = float(cotacao_euro)

#ouro
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)


#Atualizar preço de compra 
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
#Atualizar preço de venda
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
                   
display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de venda,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1447,5144.648553,1.40,7202.507974,7202.507974
1,Carro Renault,4500.00,Euro,5.3370,24016.500000,2.00,48033.000000,48033.000000
2,Notebook Dell,899.99,Dólar,5.1447,4630.178553,1.70,7871.303540,7871.303540
3,IPhone,799.00,Dólar,5.1447,4110.615300,1.70,6988.046010,6988.046010
4,Carro Fiat,3000.00,Euro,5.3370,16011.000000,1.90,30420.900000,30420.900000
5,Celular Xiaomi,480.48,Dólar,5.1447,2471.925456,2.00,4943.850912,4943.850912
6,Joia 20g,20.00,Ouro,301.7800,6035.600000,1.15,6940.940000,6940.940000


In [43]:
#Eliminei coluna duplicada
tabela = tabela.drop("Preço de Venda",axis=1)                   
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de venda
0,Câmera Canon,999.99,Dólar,5.1447,5144.648553,1.40,7202.507974
1,Carro Renault,4500.00,Euro,5.3370,24016.500000,2.00,48033.000000
2,Notebook Dell,899.99,Dólar,5.1447,4630.178553,1.70,7871.303540
3,IPhone,799.00,Dólar,5.1447,4110.615300,1.70,6988.046010
4,Carro Fiat,3000.00,Euro,5.3370,16011.000000,1.90,30420.900000
5,Celular Xiaomi,480.48,Dólar,5.1447,2471.925456,2.00,4943.850912
6,Joia 20g,20.00,Ouro,301.7800,6035.600000,1.15,6940.940000


In [46]:
#Salvar novos preços - passo 8
tabela.to_excel("Produtos Novos.xlsx", index=False)